In [51]:
import pandas as pd
import numpy as np
import sklearn as sk
import os

import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [10]:
BASELINE_START = "baselinestart"
BASELINE_END = "baselineend"
EASY_START = "easystart"
EASY_END = "easyend"
HARD_START = "hardstart"
HARD_END = "hardend"

In [11]:
"""Gets the row blocks for easy and hard tasks
"""
def read_data(fnirs_path, marker_path):
    fnirs_df =  pd.read_csv(fnirs_path, sep='\t', skiprows=range(4), index_col=False)
    marker_df = pd.read_csv(marker_path, sep='\t', skiprows=range(4), index_col=False)
    
    merged_df = pd.merge(fnirs_df, marker_df, on="Matlab_now", how="left")
    
    return merged_df

In [16]:
def get_row_blocks(merged_df):
    easy_start_rows = merged_df.index[merged_df.Stimulus_Label == EASY_START].tolist()
    easy_end_rows = merged_df.index[merged_df.Stimulus_Label == EASY_END].tolist()
    hard_start_rows = merged_df.index[merged_df.Stimulus_Label == HARD_START].tolist()
    hard_end_rows = merged_df.index[merged_df.Stimulus_Label == HARD_END].tolist()
    
    easy_rows = list(zip(easy_start_rows, easy_end_rows))
    hard_rows = list(zip(hard_start_rows, hard_end_rows))
    
    return (easy_rows, hard_rows)

In [17]:
"""Return subset of df determined by the indices of the row blocks
"""
def get_subsets(merged_df, row_blocks):
    tables = []
    column_names = ["Matlab_now", "A-DC1", "A-DC2", "A-DC3", "A-DC4", "A-DC5",
                    "A-DC6", "A-DC7", "A-DC8", "B-DC1", "B-DC2", "B-DC3", 
                    "B-DC4", "B-DC5", "B-DC6", "B-DC7", "B-DC8"]
    column_indices = [merged_df.columns.get_loc(c) for c in column_names]
    for row_block in row_blocks:
        df = merged_df.iloc[row_block[0]:row_block[1], column_indices]
        start_time = df.iloc[0]["Matlab_now"]
        df["Matlab_now"] = df["Matlab_now"] - start_time

        tables.append(df)
    return tables

In [18]:
"""runs polyfit on the timeseries data
    :param tables: table of blocks of easy / hard tasks
    :param difficulty: 0 - easy, 1 - hard; labeling process
    
    :return: numpy array of
    feature_row: 
        AC-1 gradient, AC-1 intercept, AC-1 mean, AC-2 gradient ... DC-8 gradient DC-8 intercept DC-8 mean difficulty
    
    
    Dictionary of key: channel
                           value: (gradient, mean, difficulty)
"""
def extract_features(tables, difficulty):
    arr = np.empty((0, 49))
    for table in tables:
        x = table["Matlab_now"].values
        cols = table.columns[1:]
        feature_row = []
        for col in cols:
            y = table[col].values
            gradient, intercept = np.poly1d(np.polyfit(x, y, 1))
            avg = table[col].mean()
            
            feature_row = feature_row + [gradient, intercept, avg]
        feature_row.append(difficulty)
        feature_row = np.array(feature_row)
        
        arr = np.vstack([arr, feature_row])
    return arr

In [19]:
"""Extract features from given dataset
    :param data_path: Directory containing the files
    
    :return: gets all the easy and hard features from a given dataset
"""
def get_features_for_dataset(data_path):
    fnirs_path = os.path.join(os.getcwd(), data_path, "fNIRSdata.txt")
    marker_path = os.path.join(os.getcwd(), data_path, "markers.txt")
    merged_df = read_data(fnirs_path, marker_path)
    easy_rows, hard_rows = get_row_blocks(merged_df)
    
    easy_tables = get_subsets(merged_df, easy_rows)
    hard_tables = get_subsets(merged_df, hard_rows)
    easy_feature_rows = extract_features(easy_tables, 0)
    hard_feature_rows = extract_features(hard_tables, 1)
    
    features = np.vstack([easy_feature_rows, hard_feature_rows])

    return features

In [20]:
features_902 = get_features_for_dataset("/Users/sjjin/workspace/hci_lab/data/S902/2015-02-26_11-24-48-120")
features_903 = get_features_for_dataset("/Users/sjjin/workspace/hci_lab/data/S903/2015-02-27_13-20-42-120")
features_904 = get_features_for_dataset("/Users/sjjin/workspace/hci_lab/data/S904/2015-02-27_15-30-27-120")
features_905 = get_features_for_dataset("/Users/sjjin/workspace/hci_lab/data/S905/2015-03-02_13-14-35-120")
features_906 = get_features_for_dataset("/Users/sjjin/workspace/hci_lab/data/S906/2015-03-05_11-17-38-120")

train_set = np.vstack([features_902, features_903, features_904, features_905])
test_set = features_906

In [31]:
train_x = train_set[:, :-1]
train_y = train_set[:, -1]
test_x = test_set[:, :-1]
test_y = test_set[:, -1]

In [40]:
train_x.shape

(88, 48)

In [41]:
train_y.shape

(88,)

In [35]:
logisticRegr = LogisticRegression()
logisticRegr.fit(train_x, train_y)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [58]:
log_pred = logisticRegr.predict(test_x)
accuracy_score(test_y, log_pred)

0.47619047619047616

In [52]:
clf=RandomForestClassifier(n_estimators=100)
clf.fit(train_x, train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [55]:
y_pred = clf.predict(test_x)

In [56]:
accuracy_score(test_set_y, y_pred)

0.7619047619047619